In [1]:

##import libraries
from tinyimagenet import TinyImageNet
from pathlib import Path
import torch
from torchvision import models
import pandas as pd
import os
import torch.utils.data as data
#from torchvision.models._api import WeightsEnum
#from torch.hub import load_state_dict_from_url
import numpy as np
import tmeasures as tm
### para mostrar la trasnformacion
from tinyimagenet import TinyImageNet
from torchvision import transforms as T
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt

import seaborn as sns
from pylab import *
import sys

sns.set()
#%%

#sys.path.append('./Notebooks/')
#torch.cuda.set_per_process_memory_fraction(0.5, device=0)


In [2]:
# load transformations
import transformaciones as tr

rotation_transforms = tr.rotation_transforms()
translation_transforms = tr.translation_transforms()
scale_transforms = tr.scale_transforms()
perspective_transforms = tr.perspective_transforms()
brightness_transforms = [tr.brightness_transforms(factor) for factor in tr.brightness_parameters]
contrast_transformations = [tr.contrast_transforms(alpha) for alpha in tr.contrast_list]
grayscale_transformations = [tr.grayscale_transforms(alpha) for alpha in tr.grey_list]
solarize_transformations = [tr.solarize_transforms(threshold) for threshold in tr.solarization_thresholds]
posterize_transformations = [tr.posterize_transforms(alpha) for alpha in tr.posterize_list]
invertion_transformations = [tr.invertion_transforms(alpha) for alpha in tr.invertion_list]


transformation_afin = [rotation_transforms,
                       translation_transforms,
                       scale_transforms,
                       perspective_transforms,
                       brightness_transforms,
                       contrast_transformations,
                       grayscale_transformations,
                       solarize_transformations,
                       posterize_transformations,
                       invertion_transformations]


transformaciones = ['rotacion','traslacion','escala','proyeccion','brillo','contraste','escala_grises','solarizacion','posterizacion','inversion_colores']

print('import transformations')


import transformations


# Calcula la invarianza

In [3]:
main_directory = os.path.join(os.getcwd(),'models', 'ViT','weights_none')
print(main_directory)

/Users/tatibada/Documents/Tesis_Maestria_DM/servidor_facu/models/ViT/weights_none


In [4]:
class TinyImageNet(TinyImageNet):
    def __getitem__(self, index):
        x, y = super().__getitem__(index)
        return x

normalize_transform = T.Compose(
    [
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(TinyImageNet.mean,TinyImageNet.std),
    #random_ts
    ])


dataset_nolabels = TinyImageNet(root="~/.datasets/tinyimagenet/",split="train", transform=normalize_transform)

# Subsample 
N = 1000
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
indices, _ = train_test_split(np.arange(len(dataset_nolabels)), train_size=N, stratify=dataset_nolabels.targets,random_state=24)
test_inv = Subset(dataset_nolabels, indices)

print(len(test_inv))

1000


In [13]:
from timm.models import _builder

_builder._update_default_kwargs = _builder._update_default_model_kwargs

## fastervit
from fastervit import create_model
# Define fastervit-0 model with 224 x 224 resolution

model = create_model('faster_vit_0_224', 
                          pretrained=False,
                          model_path="/home/tbadaracco//faster_vit_0.pth.tar")

## Para todas las transformaciones excepto identidad

In [9]:
# Recorrer las carpetas
for root, dirs, files in os.walk(main_directory):
    # Si el directorio .ipynb_checkpoints está en la lista de directorios, elimínalo
    if '.ipynb_checkpoints' in dirs:
        dirs.remove('.ipynb_checkpoints')
    print(root)
    # Verificar si hay un archivo 'checkpoint_last.ckpt' en la lista de archivos de la carpeta actual
    for dir in dirs:
        ## tsin trasnformacion elmodelo se entreno con la capa head modificada
        #if 'transformation' in dir:
        #    continue
        #else:
        if 'transformation'in dir:

            dir_path = os.path.join(root, dir)
            model_path = os.path.join(dir_path, 'checkpoint_last.ckpt')
            print("Model Path:", model_path)

            # Definir el folder de resultados para cada combinación de modelo e imagen
            results_folder_original = os.path.join(root, 'Invariance_Results')

            # obtain transformation from directory
            directorio_padre = os.path.dirname(model_path)
            nombre_directorio = os.path.basename(directorio_padre)

            print("Transformacion con la que se entreno: ",nombre_directorio)

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(device)
            #results_path = Path("~/tm_example_pytorch/").expanduser()
            #results_path.mkdir(parents=True, exist_ok=True)

            # es para cando se entreno con transofmrciones

            tinyimagenet_classes = 200
            # model.classifier = torch.nn.Sequential(
            #     torch.nn.Dropout(p=0.2, inplace=True),
            #     torch.nn.Linear(1280, tinyimagenet_classes),
            # )

            model.head = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2, inplace=True),
            torch.nn.Linear(512, tinyimagenet_classes),  # Ajustar el tamaño según la salida de la capa anterior
            )

            # Load the model state dictionary
            checkpoint = torch.load(model_path, map_location=device)



            # If the checkpoint is a state dictionary directly, use it; otherwise, look for the 'state_dict' key
            if 'state_dict' in checkpoint:
                model_state_dict = checkpoint['state_dict']
            else:
                model_state_dict = checkpoint

        


            # Load the model state dictionary
            model.load_state_dict(model_state_dict)

            
            
            
            


            results_folder = os.path.join('Invariance_Results/vit/from_scratch/' , nombre_directorio)
            
            image_path = os.path.join(results_folder,'imagenes')
            print("Directorio donde se guardan las medidas de invarianza para ", nombre_directorio,": ",results_folder)
            

            # Verificar si el directorio existee
            if not os.path.exists(image_path):
                # Si no existe, crearlo
                os.makedirs(image_path)
                print(f"Se ha creado el directorio '{image_path}'.")

        
            model = model.to(device)
            model.eval()


            for transformacion,transf_vector in zip(transformaciones,transformation_afin):
                print("Transformacion con la que se va evaluar: ",transformacion)
                transformations = [  transf_vector[i] for i in range(len(transf_vector))]
                
                ## results path
                #results_folder = 'Resultados de medida de invarianza'
                csv_path = os.path.join(results_folder, transformacion + '.csv')
                
                # Verificar si el archivo CSV ya existe
                if os.path.exists(csv_path):
                    print(f"El archivo CSV '{csv_path}' ya existe. Se pasará al siguiente proceso.")
                    continue  # Pasa al siguiente proceso sin guardar el archivo CSV
                


                # Create an ActivationsModule from the vanilla model
                # def filter_stochastic(a):
                #     return not str(a).startswith("StochasticDepth")
                def filter_activations(layer):
                    # Ajusta este filtro según las necesidades específicas de tu modelo
                    return not str(layer).startswith("StochasticDepth") and not str(layer).startswith("Upsample")
                
                def remove_classifier(model):
                    if hasattr(model, 'classifier'):
                        print("Removing 'classifier' layer...")
                        delattr(model, 'classifier') 
                    return model

                model = remove_classifier(model)


                activations_module = tm.pytorch.AutoActivationsModule(model,filter=filter_activations)


                # Define options for computing the measure
                options = tm.pytorch.PyTorchMeasureOptions(batch_size=2, num_workers=0,model_device=device,measure_device=device,data_device = 'cpu') #,data_device="cpu"

                # Define the measure and evaluate it
                measure = tm.pytorch.NormalizedVarianceInvariance()

                print('Se definio la medida')

                measure_result:tm.pytorch.PyTorchMeasureResult = measure.eval(test_inv,transformations,activations_module,options)  ## lista de invarianzas de cada capa

                measure_result = measure_result.numpy()
            

                vec_inv = tm.pytorch.PyTorchMeasureResult.per_layer_average(measure_result)

                vec_layer = measure_result.layer_names

                df_act = pd.DataFrame({'layer_name' : vec_layer,'inv_avg':vec_inv})

                df_act.to_csv(csv_path,index= False)
                print(f'Se guardo el csv en {csv_path}')


/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none
Model Path: /home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/Without_transformation/checkpoint_last.ckpt
Transformacion con la que se entreno:  Without_transformation
cuda
Directorio donde se guardan las medidas de invarianza para  Without_transformation :  Invariance_Results/vit/from_scratch/Without_transformation
Se ha creado el directorio 'Invariance_Results/vit/from_scratch/Without_transformation/imagenes'.


/tmp/ipykernel_38962/2683005125.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Transformacion con la que se va evaluar:  rotacion
Se definio la medida


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

Se guardo el csv en Invariance_Results/vit/from_scratch/Without_transformation/posterizacion.csv
Transformacion con la que se va evaluar:  inversion_colores
Se definio la medida


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Se guardo el csv en Invariance_Results/vit/from_scratch/Without_transformation/inversion_colores.csv
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/proyeccion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/traslacion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/brillo
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/escala
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/inversion_colores
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/escala_grises
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/rotacion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/Without_transformation
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/solarizacion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/contraste
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/posterizacion


## Para transformacion Identidad

In [14]:
# Recorrer las carpetas
for root, dirs, files in os.walk(main_directory):
    # Si el directorio .ipynb_checkpoints está en la lista de directorios, elimínalo
    if '.ipynb_checkpoints' in dirs:
        dirs.remove('.ipynb_checkpoints')
    print(root)
    # Verificar si hay un archivo 'checkpoint_last.ckpt' en la lista de archivos de la carpeta actual
    for dir in dirs:
       ## tsin trasnformacion elmodelo se entreno con la capa head modificada
        if 'transformation' in dir:
            #continue
        #else:
            dir_path = os.path.join(root, dir)
            model_path = os.path.join(dir_path, 'checkpoint_last.ckpt')
            print("Model Path:", model_path)

            # Definir el folder de resultados para cada combinación de modelo e imagen
            results_folder_original = os.path.join(root, 'Invariance_Results')

            # obtain transformation from directory
            directorio_padre = os.path.dirname(model_path)
            nombre_directorio = os.path.basename(directorio_padre)

            print("Transformacion con la que se entreno: ",nombre_directorio)

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(device)
            #results_path = Path("~/tm_example_pytorch/").expanduser()
            #results_path.mkdir(parents=True, exist_ok=True)

            tinyimagenet_classes = 200
            # model.classifier = torch.nn.Sequential(
            # torch.nn.Dropout(p=0.2, inplace=True),
            #         torch.nn.Linear(1280, tinyimagenet_classes),
            #     )

            model.head = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2, inplace=True),
            torch.nn.Linear(512, tinyimagenet_classes),  # Ajustar el tamaño según la salida de la capa anterior
            )
            # Load the model state dictionary
            checkpoint = torch.load(model_path, map_location=device)


            # If the checkpoint is a state dictionary directly, use it; otherwise, look for the 'state_dict' key
            if 'state_dict' in checkpoint:
                model_state_dict = checkpoint['state_dict']
            else:
                model_state_dict = checkpoint

            # Load the model state dictionary
            model.load_state_dict(model_state_dict)

            model = model.to(device)
            model.eval()


            results_folder = os.path.join(os.getcwd(),'Invariance_Results/vit/from_scratch/' , nombre_directorio)
            
            image_path = os.path.join(results_folder,'imagenes')
            print("Directorio donde se guardan las medidas de invarianza para ", nombre_directorio,": ",results_folder)
            

            # Verificar si el directorio existee
            if not os.path.exists(image_path):
                # Si no existe, crearlo
                os.makedirs(image_path)
                print(f"Se ha creado el directorio '{image_path}'.")


            
            transformacion = 'Identidad'
            print("Transformacion con la que se va evaluar: ",transformacion)
            def identity_transform(x):
                return x

            transformations = [identity_transform]
            
            ## results path
            #results_folder = 'Resultados de medida de invarianza'
            csv_path = os.path.join(results_folder, transformacion + '.csv')
            
            # Verificar si el archivo CSV ya existe
            if os.path.exists(csv_path):
                print(f"El archivo CSV '{csv_path}' ya existe. Se pasará al siguiente proceso.")
                continue  # Pasa al siguiente proceso sin guardar el archivo CSV


            def filter_activations(layer):
                # Ajusta este filtro según las necesidades específicas de tu modelo
                return not str(layer).startswith("StochasticDepth") and not str(layer).startswith("Upsample")
            
            def remove_classifier(model):
                if hasattr(model, 'classifier'):
                    print("Removing 'classifier' layer...")
                    delattr(model, 'classifier') 
                return model

            model = remove_classifier(model)


            activations_module = tm.pytorch.AutoActivationsModule(model,filter=filter_activations)


            # Define options for computing the measure
            options = tm.pytorch.PyTorchMeasureOptions(batch_size=2, num_workers=0,model_device=device,measure_device=device,data_device = 'cpu') #,data_device="cpu"

            # Define the measure and evaluate it
            measure = tm.pytorch.NormalizedVarianceInvariance()

            print('Se definio la medida')

            measure_result:tm.pytorch.PyTorchMeasureResult = measure.eval(test_inv,transformations,activations_module,options)  ## lista de invarianzas de cada capa

            measure_result = measure_result.numpy()
        

            vec_inv = tm.pytorch.PyTorchMeasureResult.per_layer_average(measure_result)

            vec_layer = measure_result.layer_names

            df_act = pd.DataFrame({'layer_name' : vec_layer,'inv_avg':vec_inv})

            df_act.to_csv(csv_path,index= False)
            print(f'Se guardo el csv en {csv_path}')


/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none
Model Path: /home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/Without_transformation/checkpoint_last.ckpt
Transformacion con la que se entreno:  Without_transformation
cuda
Directorio donde se guardan las medidas de invarianza para  Without_transformation :  /home/tbadaracco/tb_env/servidor_facu/Invariance_Results/vit/from_scratch/Without_transformation
Transformacion con la que se va evaluar:  Identidad
Se definio la medida


/tmp/ipykernel_38962/4154077270.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Se guardo el csv en /home/tbadaracco/tb_env/servidor_facu/Invariance_Results/vit/from_scratch/Without_transformation/Identidad.csv
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/proyeccion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/traslacion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/brillo
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/escala
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/inversion_colores
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/escala_grises
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/rotacion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/Without_transformation
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/solarizacion
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/contraste
/home/tbadaracco/tb_env/servidor_facu/models/ViT/weights_none/posterizacion


In [ ]:
## estudiar las layers para crear los graficos
csv = '/home/tbadaracco/tb_env/servidor_facu/Invariance_Results/vit/transfer_learning/posterizacion/Identidad.csv'
df = pd.read_csv(csv)
df

In [ ]:
sep='/'
s=df.layer_name.str.count(sep)
df_layer_split=((s.max()-s).map(lambda x : x*sep)+df.layer_name).str.split(sep,expand=True)
df_layer_split

In [ ]:
df_layer_split[1].value_counts()

In [ ]:
df_layer_split.iloc[21:40]

In [ ]:
df_layer_split.loc[df_layer_split[5] == 'levels',5] = df_layer_split[6]


In [ ]:
df_layer_split.loc[df_layer_split[5] == 'attn',]

In [ ]:
# Función que busca el término 'FasterViTLayer_n' en las columnas y extrae el valor
def find_layer(row):
    # Combinar los valores de todas las columnas en una sola cadena
    combined = ' '.join(row.values.astype(str))
    # Buscar por el patrón 'FasterViTLayer_' y extraer el valor
    for value in combined.split():
        if "FasterViTLayer_" in value:
            return value
    return np.nan  # Devolver NaN si el patrón no se encuentra

# Aplicar la función a cada fila del DataFrame
df_layer_split['stage'] = df_layer_split.apply(find_layer, axis=1)

# Función para transformar el valor de 'FasterViTLayer_n' a 'Stage n+1'
def transform_stage(value):
    if pd.isna(value):
        return np.nan
    if 'FasterViTLayer_' in value:
        # Extraer el número n de la cadena 'FasterViTLayer_n'
        number = int(value.split('_')[-1])
        # Incrementar el número en 1 y formatear a 'Stage n+1'
        return f'Stage {number + 1}'
    return value

# Aplicar la función a la columna 'stage'
df_layer_split['stage'] = df_layer_split['stage'].apply(transform_stage)
df_layer_split

In [ ]:
df_layer_split.iloc[0:8]

In [ ]:
 #Encontrar el primer índice donde 'stage' es 'Stage 1'
first_stage_1_index = df_layer_split[df_layer_split['stage'] == 'Stage 1'].index.min()

# Si hay un 'Stage 1' en el DataFrame
if pd.notna(first_stage_1_index):
    # Establecer 'Stage 0' para todos los índices anteriores a 'Stage 1'
    df_layer_split.loc[:first_stage_1_index - 1, 'stage'] = 'Stage 0'

df_layer_split.loc[df_layer_split['stage'].isna(), 'stage'] = 'Head'

In [ ]:
df_layer_split.loc[df_layer_split.stage == 'Stage 3',8]

# Crea los graficos

In [4]:
# funciones necesarias

# Función que busca el término 'FasterViTLayer_n' en las columnas y extrae el valor
def find_layer(row):
    # Combinar los valores de todas las columnas en una sola cadena
    combined = ' '.join(row.values.astype(str))
    # Buscar por el patrón 'FasterViTLayer_' y extraer el valor
    for value in combined.split():
        if "FasterViTLayer_" in value:
            return value
    return np.nan  # Devolver NaN si el patrón no se encuentra



# Función para transformar el valor de 'FasterViTLayer_n' a 'Stage n+1'
def transform_stage(value):
    if pd.isna(value):
        return np.nan
    if 'FasterViTLayer_' in value:
        # Extraer el número n de la cadena 'FasterViTLayer_n'
        number = int(value.split('_')[-1])
        # Incrementar el número en 1 y formatear a 'Stage n+1'
        return f'Stage {number + 1}'
    return value

In [5]:
def clean_df(df):

    sep='/'
    s=df.layer_name.str.count(sep)
    df_layer_split=((s.max()-s).map(lambda x : x*sep)+df.layer_name).str.split(sep,expand=True)

    # Aplicar la función a cada fila del DataFrame
    df_layer_split['stage'] = df_layer_split.apply(find_layer, axis=1)

    # Aplicar la función a la columna 'stage'
    df_layer_split['stage'] = df_layer_split['stage'].apply(transform_stage)
    
    #Encontrar el primer índice donde 'stage' es 'Stage 1'
    first_stage_1_index = df_layer_split[df_layer_split['stage'] == 'Stage 1'].index.min()

    # Si hay un 'Stage 1' en el DataFrame
    if pd.notna(first_stage_1_index):
        # Establecer 'Stage 0' para todos los índices anteriores a 'Stage 1'
        df_layer_split.loc[:first_stage_1_index - 1, 'stage'] = 'Stage 0'

    df_layer_split.loc[df_layer_split['stage'].isna(), 'stage'] = 'Head'        

    # cat_layer_order = CategoricalDtype(
    #     ['Normalize_0','Conv2d_0', 'BatchNorm2d_1', 'SiLU_2', 'avgpool', 'fc1','activation', 'fc2', 'scale_activation', 'Dropout_0', 'Linear_1'],
    # ordered=True
    # )

    # cat_layer_order_2 = CategoricalDtype(
    #     ['Normalization', 'Conv2dNormActivation_0', 'Conv2dNormActivation_1', 'SqueezeExcitation_1', 'SqueezeExcitation_2', 'Conv2dNormActivation_2', 'Conv2dNormActivation_3', 'Conv2dNormActivation_8', 'pre-classifier', 'classifier'],
    #     ordered=True
    # )

    # # Convertir columnas al tipo categórico
    # df_layer_split[4] = df_layer_split[4].astype(cat_layer_order)
    # df_layer_split[3] = df_layer_split[3].astype(cat_layer_order_2)
    
    #df_layer_split.sort_values(by = [0,1,2,3,4], inplace = True)

    #df_layer_split.rename(columns={0:'Sequential'},inplace=True)

    df_layer_split.reset_index(inplace=True)
    df_join = pd.concat([df.loc[:,['layer_name', 'inv_avg']], df_layer_split], axis=1)


    df_join.sort_values(by = 'index', inplace = True)
    df_join.reset_index(inplace=True,drop=True)

    # df_join['layer'] = df_join[4].str.split('_').str[0]
    # df_join['layer'] = df_join['layer'] + '_' + df_join.index.astype(str)
    # df_join = df_join.loc[~df_join.Sequential.isna()]
    # df_join.reset_index(inplace=True,drop=True)
    #print(df_join.shape)
    
    return df_join



In [6]:
df_clean = clean_df(df)
df_clean

NameError: name 'df' is not defined

In [ ]:
main_directory = '/home/tbadaracco/tb_env/servidor_facu/Invariance_Results/vit/from_scratch'
for root, dirs, files in os.walk(main_directory):
    for file in files:
        if file.endswith(".csv") and 'clean' not in file:
            file_path = os.path.join(root, file)
            print(file_path)
            # Leer el archivo CSV
            df = pd.read_csv(file_path)
            #print(df.head())
            # Limpiar el DataFrame
            df_clean = clean_df(df)
            # Obtener el nombre del archivo sin la extensión
            file_name, file_ext = os.path.splitext(file)
            # Construir el nuevo nombre de archivo con "_clean" añadido antes de la extensión
            new_file_name = f"{file_name}_clean{file_ext}"
            # Construir la ruta de destino para guardar el archivo CSV limpio
            csv_path = os.path.join(root, new_file_name)
            # Guardar el DataFrame limpio como un nuevo archivo CSV
            df_clean.to_csv(csv_path, index=False)
            print(f'Se guardó el archivo CSV limpio en {csv_path}')
            


In [ ]:
def grafico_completo(df_join,entrenado_transformacion,transformacion):
    
    #cmap = plt.cm.get_cmap('plasma', 9)    # PiYG
    cmap = plt.colormaps['plasma'].resampled(9)
    color = []
    for i in range(cmap.N):
        rgba = cmap(i)
        # rgb2hex accepts rgb or rgba

        color.append(matplotlib.colors.rgb2hex(rgba))

    df_join = df_join.iloc[1:,:]
    df_join.reset_index(inplace = True,drop=True)
    keys = df_join['stage'].unique()
    colours =  dict(zip(keys, color))


    plt.figure(figsize=(35,8))
    for i in range(len(df_join)-1):
        x = df_join['layer_name'][i], df_join['layer_name'][i+1]
        y = df_join['inv_avg'][i], df_join['inv_avg'][i+1]
        c = colours[df_join['stage'][i]]
        ax = sns.lineplot(x=x, y=y,color = c,linewidth = 5)
        mod_layer = df_join['layer_name'].str.split('_').str[0]
        ax.set(xlim=(0, 15))
        ax.set_xticks(range(len(df_join)), labels=mod_layer)
        ax.tick_params(axis='x', rotation=90 ,which='major', pad=15)
        ax.set_xlabel("Capas")
        ax.set_ylabel("Varianza Normalizada")
        ax.set_title(f'Invarianza por Stage FasterViT re-entrenado con {entrenado_transformacion} y evaluado en {transformacion}')

    # Obtener la figura actual y guardarla en una variable
    fig = plt.gcf()
    return fig


In [ ]:
main_directory = '/home/tbadaracco/tb_env/servidor_facu/Invariance_Results/vit/from_scratch'
print(main_directory)
## Aplico primer gráfico
for root, dirs, files in os.walk(main_directory):
    # Filtrar solo carpetas, ignorar archivos CSV en esta iteración
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        
        # Recorrer los archivos dentro de cada carpeta
        for sub_root, sub_dirs, sub_files in os.walk(dir_path):
            for file in sub_files:
                if file.endswith("_clean.csv") and 'checkpoint' not in file:
                    file_path = os.path.join(sub_root, file)

                    # Leer el archivo clean CSV
                    df = pd.read_csv(file_path)

                    # Obtener el nombre del archivo sin la extensión
                    file_name, file_ext = os.path.splitext(file)
                    file_name = file_name[:-len("_clean")]
                    directorio_padre = os.path.dirname(file_path)
                    nombre_directorio = os.path.basename(directorio_padre)

                    # Verificar si el gráfico ya ha sido guardado
                    imagenes_dir = os.path.join(sub_root, 'imagenes')
                    imagen_path = os.path.join(imagenes_dir, f'{file_name}_plot_1.png')
                    if os.path.exists(imagen_path):
                        print("El gráfico ya existe, paso al siguiente")
                        continue

                    # Crear gráfico
                    fig = grafico_completo(df, nombre_directorio, file_name)
                    fig.savefig(imagen_path)
                    print(f'Se guardó el gráfico en {imagen_path}')

In [ ]:
cmap = plt.colormaps['plasma'].resampled(9)    # PiYG
color = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    color.append(matplotlib.colors.rgb2hex(rgba))

In [ ]:
def grafico_por_estados(df_join,entrenado_transformacion,transformacion):
    
    df_join = df_join.iloc[1:,:]
    df_join.reset_index(inplace = True,drop=True)
    keys = df_join['Sequential'].unique()
    colours =  dict(zip(keys, color))
    #  Categorical Data
    a = 3  # number of rows
    b = 3  # number of columns
    c = 1  # initialize plot counter

    fig = plt.figure(figsize=(20,8))

    for seq in df_join.Sequential.unique():
        plt.subplot(a, b, c)
        df = df_join.loc[df_join.Sequential == seq,]
        col = colours[seq]
        g = sns.lineplot(data = df, x='layer', y='inv_avg',color = col,linewidth = 2,marker = 'o')
        mod_layer = df['layer'].str.split('_').str[0]
        #g.set(xlim=(0, 15))
        g.set_xticks(range(len(df)), labels=mod_layer)
        g.tick_params(axis='x', rotation=90 )
        c = c + 1
        plt.xticks(df['layer'][::1])
        g.set_xlabel("Capas")
        g.set_ylabel("Varianza Normalizada")
        titulo = 'Estado ' + str(c - 1)
        plt.title(titulo)
        titulo_general = f'Invarianza por capa con transformación re-entrenado con {entrenado_transformacion} y evaluado en {transformacion}'
        plt.suptitle(titulo_general, fontsize=16)
        plt.tight_layout()

    fig = plt.gcf()
    return fig

In [ ]:
for root, dirs, files in os.walk(main_directory):
    # Filtrar solo carpetas, ignorar archivos CSV en esta iteración
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        #print(dir_path)
        for sub_root, sub_dirs, sub_files in os.walk(dir_path):
            for file in sub_files:
                #print(file)
                if file.endswith("_clean.csv") and 'checkpoint' not in file:
                    file_path = os.path.join(dir_path, file)

                    # Leer el archivo clean CSV
                    df = pd.read_csv(file_path)
                    # Obtener el nombre del archivo sin la extensión
                    file_name, file_ext = os.path.splitext(file)
                    file_name = file_name[:-len("_clean")]
                    directorio_padre = os.path.dirname(file_path)
                    nombre_directorio = os.path.basename(directorio_padre)
                    # Verificar si el gráfico ya ha sido guardado
                    imagenes_dir = os.path.join(dir_path,'imagenes')
                    imagen_path = os.path.join(imagenes_dir, f'{file_name}_plot_2.png')
                    if os.path.exists(imagen_path):
                        print("el grafico ya existe paso al sgte")
                        continue
                        # Saltar este archivo si ya ha sido procesado
                    #creo grafico
                    fig = grafico_por_estados(df,nombre_directorio,file_name)
                    fig.savefig(imagen_path)
                    print(f'Se guardó el gráfico en {imagen_path}')

In [ ]:
#  quito los bloques SE para analizar la varianza sin esos bloques xq no son secuenciales
bloque_se = ['avgpool',
'fc1',
'activation',
'fc2',
'scale_activation']

In [ ]:
def grafico_sinbloqueSE_completo(df_join,entrenado_transformacion,transformacion):
    df1 = df_join.loc[~(df_join['4'].isin(bloque_se))]
    df2 = df_join.loc[(df_join['4'].isin(bloque_se)) & (df_join.Sequential == 'Sequential_8')]
    df = pd.concat([df1,df2],axis = 0)
    df = df.loc[df.Sequential != 'Normalization',]
    df.reset_index(inplace=True,drop=True)
    print(df.shape)

    keys = df['Sequential'].unique()
    colours =  dict(zip(keys, color))


    plt.figure(figsize=(35,8))
    for i in range(len(df)-1):
        x = df['layer'][i], df['layer'][i+1]
        y = df['inv_avg'][i], df['inv_avg'][i+1]
        c = colours[df['Sequential'][i]]
        ax = sns.lineplot(x=x, y=y,color = c,linewidth = 5,marker = 'o')
        mod_layer = df['layer'].str.split('_').str[0]
        ax.set(xlim=(0, 15))
        ax.set_xticks(range(len(df)), labels=mod_layer)
        ax.tick_params(axis='x', rotation=90 ,which='major', pad=15)
        ax.set_xlabel("Capas")
        ax.set_ylabel("Varianza Normalizada")
        ax.set_title(f'Invarianza por capa EfficientNetB0 sin bloques SE y re-entrenado con {entrenado_transformacion} y evaluado en {transformacion}')

    fig = plt.gcf()
    return fig
        #plt.savefig(os.path.join(image_path , 'invarianza sin bloque SE-' + transformacion + '.png'))

In [ ]:
for root, dirs, files in os.walk(main_directory):
    # Filtrar solo carpetas, ignorar archivos CSV en esta iteración
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        
        # Recorrer los archivos dentro de cada carpeta
        for sub_root, sub_dirs, sub_files in os.walk(dir_path):
            for file in sub_files:
                #print(files)
        
                if file.endswith("_clean.csv") and 'checkpoint' not in file:
                    file_path = os.path.join(dir_path, file)

                    # Leer el archivo clean CSV
                    df = pd.read_csv(file_path)
                    # Obtener el nombre del archivo sin la extensión
                    file_name, file_ext = os.path.splitext(file)
                    file_name = file_name[:-len("_clean")]
                    directorio_padre = os.path.dirname(file_path)
                    nombre_directorio = os.path.basename(directorio_padre)
                    # Verificar si el gráfico ya ha sido guardado
                    imagenes_dir = os.path.join(dir_path,'imagenes')
                    imagen_path = os.path.join(imagenes_dir, f'{file_name}_plot_3.png')
                    if os.path.exists(imagen_path):
                        print("el grafico ya existe paso al sgte")
                        continue
                        # Saltar este archivo si ya ha sido procesado
                    #creo grafico
                    fig = grafico_sinbloqueSE_completo(df,nombre_directorio,file_name)
                    fig.savefig(imagen_path)
                    print(f'Se guardó el gráfico en {imagen_path}')

In [ ]:
def grafico_sinbloqueSE_estado(df_join,entrenado_transformacion,transformacion):
    df1 = df_join.loc[~(df_join['4'].isin(bloque_se))]
    df2 = df_join.loc[(df_join['4'].isin(bloque_se)) & (df_join.Sequential == 'Sequential_8')]
    df = pd.concat([df1,df2],axis = 0)
    df = df.loc[df.Sequential != 'Normalization',]
    df.reset_index(inplace=True,drop=True)
    
    print(df.shape)


    keys = df['Sequential'].unique()
    colours =  dict(zip(keys, color))
        #  Categorical Data
    a = 3  # number of rows
    b = 3  # number of columns
    c = 1  # initialize plot counter

    fig = plt.figure(figsize=(20,10))

    for seq in df.Sequential.unique():
        plt.subplot(a, b, c)
        df_fil = df.loc[df.Sequential == seq,]
        col = colours[seq]
        g = sns.lineplot(data = df_fil, x='layer', y='inv_avg',color = col,linewidth = 2,marker = 'o')
        mod_layer = df_fil['layer'].str.split('_').str[0]
        #g.set(xlim=(0, 15))
        g.set_xticks(range(len(df_fil)), labels=mod_layer)
        g.tick_params(axis='x', rotation=90 )
        c = c + 1
        plt.xticks(df_fil['layer'][::1])
        g.set_xlabel("Capas")
        g.set_ylabel("Varianza Normalizada")
        titulo = 'Estado ' + str(c - 1)
        plt.title(titulo)
        titulo_general = f'Invarianza por capa EfficienNetB0 sin bloques SE re-entrenado con transformación {entrenado_transformacion} y evaluado con {transformacion}'
        plt.suptitle(titulo_general, fontsize=16)
    
    plt.tight_layout()

    fig = plt.gcf()
    return fig

In [ ]:
for root, dirs, files in os.walk(main_directory):
    # Filtrar solo carpetas, ignorar archivos CSV en esta iteración
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        
        # Recorrer los archivos dentro de cada carpeta
        for sub_root, sub_dirs, sub_files in os.walk(dir_path):
            for file in sub_files:
                if file.endswith("_clean.csv") and 'checkpoint' not in file:
                    file_path = os.path.join(dir_path, file)

                    # Leer el archivo clean CSV
                    df = pd.read_csv(file_path)
                    # Obtener el nombre del archivo sin la extensión
                    file_name, file_ext = os.path.splitext(file)
                    file_name = file_name[:-len("_clean")]
                    directorio_padre = os.path.dirname(file_path)
                    nombre_directorio = os.path.basename(directorio_padre)
                    # Verificar si el gráfico ya ha sido guardado
                    imagenes_dir = os.path.join(dir_path,'imagenes')
                    imagen_path = os.path.join(imagenes_dir, f'{file_name}_plot_4.png')
                    fig = grafico_sinbloqueSE_estado(df,nombre_directorio,file_name)
                    fig.savefig(imagen_path)
                    print(f'Se guardó el gráfico en {imagen_path}')

In [ ]:
def grafico_boxplot(df_join,entrenado_transformacion,transformacion):
    df1 = df_join.loc[~(df_join['4'].isin(bloque_se))]
    df2 = df_join.loc[(df_join['4'].isin(bloque_se)) & (df_join.Sequential == 'Sequential_8')]
    df = pd.concat([df1,df2],axis = 0)
    df = df.loc[df.Sequential != 'Normalization',]
    df.reset_index(inplace=True,drop=True)
    

    plt.figure(figsize=(10, 6))
    ##boxplot
    df['Estado'] = df['Sequential'].replace({
        'Sequential_0': 'Estado 1',
        'Sequential_1': 'Estado 2',
        'Sequential_2': 'Estado 3',
        'Sequential_3': 'Estado 4',
        'Sequential_4': 'Estado 5',
        'Sequential_5': 'Estado 6',
        'Sequential_6': 'Estado 7',
        'Sequential_7': 'Estado 8',
        'Sequential_8': 'Estado 9'
    })

    keys = df['Sequential'].unique()
    colours =  dict(zip(keys, color))
    
    # Crear una paleta de colores personalizada basada en el diccionario de colores
    custom_palette = sns.color_palette([colours[val] for val in df['Sequential'].unique()])

    # Crear un gráfico de caja utilizando seaborn con el parámetro "hue" y sin leyenda
    g = sns.boxplot(data=df, x='inv_avg', y='Estado', palette=custom_palette, dodge=False)

    # Establecer etiquetas de los ejes x e y
    g.set_xlabel("Varianza Normalizada")
    g.set_ylabel("Estado")

    # Establecer el título del gráfico
    titulo = f'Invarianza por capa EfficienNetB0 sin bloques SE re-entrenado con transformación {entrenado_transformacion} y evaluado con {transformacion}'
    plt.title(titulo)

    fig = plt.gcf()
    return fig


In [ ]:
for root, dirs, files in os.walk(main_directory):
    # Filtrar solo carpetas, ignorar archivos CSV en esta iteración
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        
        # Recorrer los archivos dentro de cada carpeta
        for sub_root, sub_dirs, sub_files in os.walk(dir_path):
            for file in sub_files:
                if file.endswith("_clean.csv") and 'checkpoint' not in file:
                    file_path = os.path.join(dir_path, file)

                    # Leer el archivo clean CSV
                    df = pd.read_csv(file_path)
                    # Obtener el nombre del archivo sin la extensión
                    file_name, file_ext = os.path.splitext(file)
                    file_name = file_name[:-len("_clean")]
                    directorio_padre = os.path.dirname(file_path)
                    nombre_directorio = os.path.basename(directorio_padre)
                    # Verificar si el gráfico ya ha sido guardado
                    imagenes_dir = os.path.join(dir_path,'imagenes')
                    imagen_path = os.path.join(imagenes_dir, f'{file_name}_plot_5.png')
                    if os.path.exists(imagen_path):
                        print("el grafico ya existe paso al sgte")
                        continue
                        # Saltar este archivo si ya ha sido procesado
                    fig = grafico_boxplot(df,nombre_directorio,file_name)
                    fig.savefig(imagen_path)
                    print(f'Se guardó el gráfico en {imagen_path}')

# Junto todos los csv para poder sacar metricas de la invarianza

In [11]:
main_directory = '/Users/tatibada/Documents/Tesis_Maestria_DM/servidor_facu/Invariance_Results/vit/from_scratch'

In [12]:
df_complete = pd.DataFrame()
for root, dirs, files in os.walk(main_directory):
    # Filtrar solo carpetas, ignorar archivos CSV en esta iteración
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        
        # Recorrer los archivos dentro de cada carpeta
        for sub_root, sub_dirs, sub_files in os.walk(dir_path):
            for file in sub_files:
                if file.endswith("_clean.csv") and 'checkpoint' not in file:
                    file_path = os.path.join(dir_path, file)

                    # Leer el archivo clean CSV
                    df = pd.read_csv(file_path)
                    # Obtener el nombre del archivo sin la extensión
                    file_name, file_ext = os.path.splitext(file)
                    file_name = file_name[:-len("_clean")]
                    directorio_padre = os.path.dirname(file_path)
                    nombre_directorio = os.path.basename(directorio_padre)
                    
                    ## agrego columnas entrenado e invarianza
                    df['Training'] = nombre_directorio
                    df['Eval'] = file_name
                    ## concateno
                    df_complete = pd.concat([df_complete,df])

In [8]:
os.getcwd()

'/Users/tatibada/Documents/Tesis_Maestria_DM/servidor_facu'

In [13]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39895 entries, 0 to 338
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   layer_name  39895 non-null  object 
 1   inv_avg     39895 non-null  float64
 2   index       39895 non-null  int64  
 3   0           0 non-null      float64
 4   1           6018 non-null   object 
 5   2           12036 non-null  object 
 6   3           28084 non-null  object 
 7   4           38232 non-null  object 
 8   5           38704 non-null  object 
 9   6           39412 non-null  object 
 10  7           39552 non-null  object 
 11  8           39895 non-null  object 
 12  stage       39895 non-null  object 
 13  Training    39895 non-null  object 
 14  Eval        39895 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 4.9+ MB


In [14]:
df_complete.head()

,layer_name,inv_avg,index,0,1,2,3,4,5,6,7,8,stage,Training,Eval
0,/patch_embed/proj,0.765733,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,proj,Stage 0,inversion_colores,posterizacion
1,/patch_embed/conv_down/Conv2d_0,0.646632,1,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,Conv2d_0,Stage 0,inversion_colores,posterizacion
2,/patch_embed/conv_down/BatchNorm2d_1,0.646632,2,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,BatchNorm2d_1,Stage 0,inversion_colores,posterizacion
3,/patch_embed/conv_down/ReLU_2,0.627872,3,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,ReLU_2,Stage 0,inversion_colores,posterizacion
4,/patch_embed/conv_down/Conv2d_3,0.651827,4,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,Conv2d_3,Stage 0,inversion_colores,posterizacion


In [15]:
path = os.path.join(main_directory,'df_complete.csv')
print(path)
df_complete.to_csv(path, index=False)

/Users/tatibada/Documents/Tesis_Maestria_DM/servidor_facu/Invariance_Results/vit/from_scratch/df_complete.csv


In [ ]:
df_complete.head()

In [16]:
df_complete.Training.value_counts()

Without_transformation    3729
proyeccion                3718
posterizacion             3718
escala_grises             3718
brillo                    3718
rotacion                  3718
escala                    3718
traslacion                3718
inversion_colores         3380
contraste                 3380
solarizacion              3380
Name: Training, dtype: int64

In [17]:
df_complete.Eval.value_counts()

posterizacion        3719
contraste            3719
escala               3719
proyeccion           3719
escala_grises        3719
rotacion             3719
inversion_colores    3719
solarizacion         3719
brillo               3719
traslacion           3719
Identidad            2705
Name: Eval, dtype: int64

In [18]:
df_complete.reset_index(inplace=True,drop=True)
df_complete

,layer_name,inv_avg,index,0,1,2,3,4,5,6,7,8,stage,Training,Eval
0,/patch_embed/proj,0.765733,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,proj,Stage 0,inversion_colores,posterizacion
1,/patch_embed/conv_down/Conv2d_0,0.646632,1,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,Conv2d_0,Stage 0,inversion_colores,posterizacion
2,/patch_embed/conv_down/BatchNorm2d_1,0.646632,2,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,BatchNorm2d_1,Stage 0,inversion_colores,posterizacion
3,/patch_embed/conv_down/ReLU_2,0.627872,3,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,ReLU_2,Stage 0,inversion_colores,posterizacion
4,/patch_embed/conv_down/Conv2d_3,0.651827,4,NaN,NaN,NaN,NaN,NaN,NaN,patch_embed,conv_down,Conv2d_3,Stage 0,inversion_colores,posterizacion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39890,/levels/FasterViTLayer_3/blocks/HAT_4/mlp/drop,0.782639,334,NaN,NaN,NaN,levels,FasterViTLayer_3,blocks,HAT_4,mlp,drop,Stage 4,Without_transformation,traslacion
39891,/norm,0.407935,335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,norm,Head,Without_transformation,traslacion
39892,/avgpool,0.272076,336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,avgpool,Head,Without_transformation,traslacion
39893,/head/Dropout_0,0.272076,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,head,Dropout_0,Head,Without_transformation,traslacion


In [19]:
df_complete.Training.unique()

array(['inversion_colores', 'proyeccion', 'posterizacion', 'contraste',
       'escala_grises', 'solarizacion', 'brillo', 'rotacion', 'escala',
       'traslacion', 'Without_transformation'], dtype=object)

In [ ]:
import re
# Función para extraer el sufijo numérico
def extract_numeric_suffix(layer):
    match = re.search(r'_(\d+)$', layer)
    return int(match.group(1)) if match else -1

# Crear una nueva columna con el sufijo numérico
df_complete['layer_num'] = df_complete['layer'].apply(extract_numeric_suffix)

In [ ]:
order_transformation = CategoricalDtype(
    ['without_transformation','brillo', 'contraste', 'escala_grises','inversion_colores', 'posterizacion','solarizacion', 'escala','proyeccion', 'rotacion', 'traslacion'],
    ordered=True
)

# Convertir columnas al tipo categórico
df_complete['Training'] = df_complete['Training'].astype(order_transformation)
df_complete['Eval'] = df_complete['Eval'].astype(order_transformation)
df_complete.sort_values(by = ['Training','Eval','Sequential', 'layer_num'], ascending=[True, True, True, True],inplace=True)
df_complete

In [ ]:
df_complete.loc[df_complete.Sequential == 'Sequential_0']

In [ ]:
df_complete.groupby(['Training','Eval']).inv_avg.describe()

In [ ]:
df_complete.columns

In [ ]:
df_mean = df_complete.groupby(['Training', 'Eval','Sequential']).agg({'inv_avg':'mean'}).reset_index()

In [ ]:
df_mean

In [ ]:
df_pivot = df_mean.pivot(index=['Training', 'Sequential'], columns=["Eval"], values="inv_avg")
df_pivot

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 25))
sns.heatmap(df_pivot, annot=False, cbar=True)

plt.title('Heatmap de promedio de Invarianza por Estado')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()

In [ ]:
df_pivot2 = df_mean.pivot_table(index=['Training'], columns='Eval', values='inv_avg', aggfunc='mean')
df_pivot2

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 6))
sns.heatmap(df_pivot2, annot=True, cbar=True)

plt.title('Heatmap de promedio de Invarianza por Estado')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()

In [ ]:
os.getcwd()

### Mismo analisis sin bloque SE

In [ ]:
#  quito los bloques SE para analizar la invarianza sin esos bloques xq no son secuenciales
bloque_se = ['avgpool',
'fc1',
'activation',
'fc2',
'scale_activation']


In [ ]:
df_complete.columns

In [ ]:
df_without_se = df_complete.loc[~df_complete['4'].isin(bloque_se)]
df_without_se.shape

In [ ]:
df_mean2 = df_without_se.groupby(['Training', 'Eval','Sequential']).agg({'inv_avg':'mean'}).reset_index()

In [ ]:
df_pivot3 = df_mean2.pivot(index=['Training', 'Sequential'], columns=["Eval"], values="inv_avg")
df_pivot3

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 25))
sns.heatmap(df_pivot3, annot=False, cbar=True)

plt.title('Heatmap de promedio de Invarianza por Estado')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()

In [ ]:
df_pivot4 = df_mean2.pivot_table(index=['Training'], columns='Eval', values='inv_avg', aggfunc='mean')
df_pivot4

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 6))
sns.heatmap(df_pivot4, annot=True, cbar=True)

plt.title('Heatmap de promedio de Invarianza por Estado')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()

### Mismo analisis de la salida de cada bloque

In [ ]:
# Función para extraer el sufijo numérico
def extract_numeric_suffix(layer):
    match = re.search(r'_(\d+)$', layer)
    return int(match.group(1)) if match else -1

# Crear una nueva columna con el sufijo numérico
df_complete['layer_num'] = df_complete['layer'].apply(extract_numeric_suffix)

# Ordenar el DataFrame por 'Training', 'Eval', 'Sequential' y 'layer_num'
df_sorted = df_complete.sort_values(by=['Training', 'Eval', 'Sequential', 'layer_num'], ascending=[True, True, True, False])

# Obtener la capa con el valor más grande en el sufijo dentro de cada grupo
df_max_layer = df_sorted.groupby(['Training', 'Eval', 'Sequential']).first().reset_index()

# Eliminar la columna auxiliar 'layer_num' si ya no es necesaria
df_max_layer = df_max_layer.drop(columns=['layer_num'])

# Mostrar el DataFrame con la capa de mayor valor en el sufijo por grupo
df_max_layer

In [ ]:
df_max_layer.loc[(df_max_layer.Training == 'brillo') & (df_max_layer.Eval == 'brillo') ]

In [ ]:
df_pivot5 = df_max_layer.pivot(index=['Training', 'Sequential'], columns=["Eval"], values="inv_avg")
df_pivot5

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 25))
sns.heatmap(df_pivot5, annot=False, cbar=True)

plt.title('Heatmap de Invarianza de la salida de cada Estado')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()

In [ ]:
df_pivot6 = df_max_layer.pivot_table(index=['Training'], columns='Eval', values='inv_avg', aggfunc='mean')
df_pivot6

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 6))
sns.heatmap(df_pivot6, annot=True, cbar=True)

plt.title('Heatmap de promedio de Invarianza de las salidas de cada estado')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()

# Accuracy Training Set

In [ ]:
df_acc = pd.read_csv('/home/tbadaracco/models/efficientnet_b0/transfer_learning/performance_metrics_final_training.csv')

In [ ]:
df_acc

In [ ]:
order_transformation = CategoricalDtype(
    ['without_transformation','brillo', 'contraste', 'escala_grises','inversion_colores', 'posterizacion','solarizacion', 'escala','proyeccion', 'rotacion', 'traslacion'],
    ordered=True
)

# Convertir columnas al tipo categórico
df_complete['Training'] = df_complete['Training'].astype(order_transformation)
df_complete['Eval'] = df_complete['Eval'].astype(order_transformation)
df_complete.sort_values(by = ['Training','Eval','Sequential', 'layer_num'], ascending=[True, True, True, True],inplace=True)
df_complete

In [ ]:
df_acc.head()

In [ ]:
df_acc_pivot = df_acc.pivot(index=['Transformation_training'], columns=["Transformation_eval"], values="Accuracy_Train")

In [ ]:
# Crear el mapa de colores
plt.figure(figsize=(10, 6))
sns.heatmap(df_acc_pivot, annot=True, cbar=True)

plt.title('Heatmap de Accuracy de training set')
plt.xlabel('Eval')
plt.ylabel('Training')
plt.show()